- Go to https://apps.twitter.com/app/new to create an app and get values for these credentials, which you'll need to provide in place of these empty string values that are defined as placeholders.


- See https://dev.twitter.com/oauth for more information on Twitter's OAuth implementation.

In [ ]:
# pip install python-twitter -> run that command in your remote machine's terminal window to install the Twitter wrapper
import twitter

# Twitter API keys go here
CONSUMER_KEY = ''
CONSUMER_SECRET = ''

OAUTH_TOKEN = ''
OAUTH_TOKEN_SECRET = ''


# let's create an instance of the twitter.Api with our credentials
api = twitter.Api(consumer_key=CONSUMER_KEY,
                  consumer_secret=CONSUMER_SECRET,
                  access_token_key=OAUTH_TOKEN,
                  access_token_secret=OAUTH_TOKEN_SECRET
)

print api

Wo0t! we've successfully used OAuth credentials to gain authorization to query Twitter's API!!!

# Searching Tweets

Let's take one of the common hashtags across trends and use it as the basis of a search query to fetch some tweets for further analysis. Here's a link to the <a class="ulink" href="http://bit.ly/1a1l398" target="\_top"><code class="literal">GET search/tweets</code> resource</a>.

In [15]:
import json

q = '#NoDAPL' 
count = 100

# See https://dev.twitter.com/rest/reference/get/search/tweets

In [67]:
statuses = api.GetSearch(term=q, count=count)

In [68]:
len(statuses)

100

In [69]:
status = statuses[0]

In [70]:
# Here are all the fields in a Tweet

status.AsDict()

{u'created_at': u'Sat Nov 26 20:52:03 +0000 2016',
 u'favorite_count': 11596,
 u'hashtags': [{u'text': u'NoDAPL'}],
 u'id': 802615926361178113,
 u'id_str': u'802615926361178113',
 u'in_reply_to_screen_name': u'SenSanders',
 u'in_reply_to_status_id': 802615712804118529,
 u'in_reply_to_user_id': 29442313,
 u'lang': u'en',
 u'retweet_count': 6805,
 u'source': u'<a href="http://twitter.com" rel="nofollow">Twitter Web Client</a>',
 u'text': u'Mr. President: Stop the Dakota Access Pipeline. #NoDAPL',
 u'urls': [],
 u'user': {u'created_at': u'Tue Apr 07 13:02:35 +0000 2009',
  u'description': u'Sen. Bernie Sanders is the longest serving independent in congressional history. Tweets ending in -B are from Bernie, and all others are from a staffer.',
  u'favourites_count': 24,
  u'followers_count': 3174497,
  u'friends_count': 1971,
  u'geo_enabled': True,
  u'id': 29442313,
  u'lang': u'en',
  u'listed_count': 17179,
  u'location': u'Vermont/DC',
  u'name': u'Bernie Sanders',
  u'profile_backgro

In [62]:
status.created_at

u'Sat Nov 26 20:52:03 +0000 2016'

In [63]:
import datetime 
datetime.datetime.strptime(status.created_at, '%a %b %d %H:%M:%S +0000 %Y').strftime('%Y-%m-%d %H')

'2016-11-26 20'

In [64]:
# Every Tweet object includes a User object that has the following fields

status.user.AsDict()

{u'created_at': u'Tue Apr 07 13:02:35 +0000 2009',
 u'description': u'Sen. Bernie Sanders is the longest serving independent in congressional history. Tweets ending in -B are from Bernie, and all others are from a staffer.',
 u'favourites_count': 24,
 u'followers_count': 3174487,
 u'friends_count': 1971,
 u'geo_enabled': True,
 u'id': 29442313,
 u'lang': u'en',
 u'listed_count': 17178,
 u'location': u'Vermont/DC',
 u'name': u'Bernie Sanders',
 u'profile_background_color': u'000000',
 u'profile_background_image_url': u'http://pbs.twimg.com/profile_background_images/514189149444661248/pkGHMpZZ.jpeg',
 u'profile_banner_url': u'https://pbs.twimg.com/profile_banners/29442313/1430854323',
 u'profile_image_url': u'http://pbs.twimg.com/profile_images/794619281271033856/Fs0QQaH7_normal.jpg',
 u'profile_link_color': u'44506A',
 u'profile_sidebar_fill_color': u'D6CCB6',
 u'profile_text_color': u'304562',
 u'screen_name': u'SenSanders',
 u'statuses_count': 15597,
 u'time_zone': u'Eastern Time (US 

In [71]:
# all users
tweet_users = [x.user.screen_name for x in statuses]
tweet_users[:20]

[u'SenSanders',
 u'ajplus',
 u'ajplus',
 u'MaxPyziur',
 u'Quickieleaks',
 u'rappublicist',
 u'emwainwright08',
 u'MaxKingsleyEh',
 u'k_nightss',
 u'yayayarndiva',
 u'canativeobt',
 u'WBirarda',
 u'BigBlack0live',
 u'purplesofa',
 u'FC_WitchParty',
 u'JamesTheFranze',
 u'chatnread',
 u'billeniumbeach',
 u'bcfontt',
 u'simonaumich']

In [72]:
# max_id

statuses[-1].id

803090431814680582

In [73]:
# In order to grab more data, we can iterate through search results by providing
# the Tweet ID of the last result returned from the previous iteration (max_id)

max_id = search_results[-1].id
statuses += api.GetSearch(term=q, count=count, max_id=max_id)

In [74]:
len(statuses)

200

In [76]:
# use a loop -> iterate multiple times to get many more tweets

num_iterations = 30

for i in range(num_iterations):
    max_id = statuses[-1].id
    statuses += api.GetSearch(term=q, count=count, max_id=max_id)

In [80]:
len(statuses)

3198

In [79]:
# save our data in pickled format - so that we don't ahve to grab it again if our machine crashes

import pickle
path = 'data/'

pickle.dump(statuses, open(path+'%s_tw.p' % q,'wb'))

This was a simple hashtag search. It is worth noting that Twitter's search API enables some more advanced queries - https://dev.twitter.com/rest/public/search

# Tweet Entities

In [81]:
status_texts = [ status.text for status in statuses ]

print json.dumps(status_texts[0:5], indent=1)

[
 "Mr. President: Stop the Dakota Access Pipeline. #NoDAPL", 
 "This veteran is shocked by the militarized police force used against #NoDAPL protesters. https://t.co/uORv4Zk4bL", 
 "#NoDAPL activists faced hypothermia, internal bleeding and more after clashing with police in sub-freezing weather. https://t.co/8KHLxXUZVZ", 
 "@HighSierraAaron @sierraclub\n#GetTheFacts\nhttps://t.co/KgrXeElVdP\n#DAPL #NoDAPL #YesDAPL https://t.co/KpvZQCHSgW", 
 "Wouldn't trust with a build up of law enforcement... \n\n#StandingRock #NoDAPL https://t.co/Dr4KAf4wp6"
]


In [89]:
screen_names = [ user_mention.screen_name 
                 for status in statuses
                     for user_mention in status.user_mentions ]

print json.dumps(screen_names[0:5], indent=1) 

[
 "HighSierraAaron", 
 "sierraclub", 
 "xeni", 
 "cj_wentz", 
 "RuthHHopkins"
]


In [91]:
hashtags = [ hashtag.text
             for status in statuses
                 for hashtag in status.hashtags ]

print json.dumps(hashtags[0:15], indent=1)

[
 "NoDAPL", 
 "NoDAPL", 
 "NoDAPL", 
 "GetTheFacts", 
 "DAPL", 
 "NoDAPL", 
 "YesDAPL", 
 "StandingRock", 
 "NoDAPL", 
 "StandingRock", 
 "nodapl", 
 "StandforStandingRock", 
 "NoDAPL", 
 "NoDAPL", 
 "NoDAPL"
]


In [92]:
# Compute a collection of all words from all tweets
words = [ w 
          for t in status_texts 
              for w in t.split() ]

print json.dumps(words[0:5], indent=1)

[
 "Mr.", 
 "President:", 
 "Stop", 
 "the", 
 "Dakota"
]


In [93]:
status_texts[0].split()

[u'Mr.',
 u'President:',
 u'Stop',
 u'the',
 u'Dakota',
 u'Access',
 u'Pipeline.',
 u'#NoDAPL']

In [94]:
len(words)

50917

## Analyzing Tweets and Tweet Entities with Frequency Analysis

From an empirical standpoint, counting observable things is the starting point for just about everything, and thus the starting point for any kind of statistical filtering or manipulation that strives to find what may be a faint signal in noisy data. Whereas we just extracted the first 5 items of each unranked list to get a feel for the data, let's now take a closer look at what's in the data by computing a frequency distribution and looking at the top 10 items in each list.

The result of the frequency distribution is a map of key/value
      pairs corresponding to terms and their frequencies, so let's make
      reviewing the results a little easier on the eyes by emitting a tabular
      format. You can install a package called <code class="literal">prettytable</code> by typing <strong class="userinput"><code>pip install prettytable</code></strong> in a terminal; this
      package provides a convenient way to emit a fixed-width tabular format
      that can be easily copied-and-pasted.

In [95]:
from collections import Counter

for item in [words, screen_names, hashtags]:
    c = Counter(item)
    print c.most_common()[:10] # top 10
    print

[(u'#NoDAPL', 2602), (u'RT', 2570), (u'the', 1114), (u'to', 1112), (u'in', 645), (u'a', 613), (u'is', 584), (u'on', 570), (u'of', 468), (u'Standing', 358)]

[(u'POTUS', 209), (u'RuthHHopkins', 123), (u'UR_Ninja', 116), (u'joshfoxfilm', 88), (u'DeanLeh', 81), (u'Heritage', 79), (u'benandjerrys', 65), (u'IENearth', 63), (u'theintercept', 56), (u'SenSanders', 55)]

[(u'NoDAPL', 2820), (u'StandingRock', 389), (u'StandWithStandingRock', 262), (u'WaterIsLife', 200), (u'nodapl', 156), (u'NODAPL', 152), (u'standingrock', 145), (u'noDAPL', 108), (u'DAPL', 82), (u'MniWiconi', 56)]



In [97]:
from prettytable import PrettyTable

for label, data in (('Word', words), 
                    ('Screen Name', screen_names), 
                    ('Hashtag', hashtags)):
    pt = PrettyTable(field_names=[label, 'Count']) 
    c = Counter(data)
    [ pt.add_row(kv) for kv in c.most_common()[:10] ]
    pt.align[label], pt.align['Count'] = 'l', 'r' # Set column alignment
    print pt

+----------+-------+
| Word     | Count |
+----------+-------+
| #NoDAPL  |  2602 |
| RT       |  2570 |
| the      |  1114 |
| to       |  1112 |
| in       |   645 |
| a        |   613 |
| is       |   584 |
| on       |   570 |
| of       |   468 |
| Standing |   358 |
+----------+-------+
+--------------+-------+
| Screen Name  | Count |
+--------------+-------+
| POTUS        |   209 |
| RuthHHopkins |   123 |
| UR_Ninja     |   116 |
| joshfoxfilm  |    88 |
| DeanLeh      |    81 |
| Heritage     |    79 |
| benandjerrys |    65 |
| IENearth     |    63 |
| theintercept |    56 |
| SenSanders   |    55 |
+--------------+-------+
+-----------------------+-------+
| Hashtag               | Count |
+-----------------------+-------+
| NoDAPL                |  2820 |
| StandingRock          |   389 |
| StandWithStandingRock |   262 |
| WaterIsLife           |   200 |
| nodapl                |   156 |
| NODAPL                |   152 |
| standingrock          |   145 |
| noDAPL        

Used code and examples from Mining the Social Web, 2nd Edition - https://rawgit.com/ptwobrussell/Mining-the-Social-Web-2nd-Edition/master/ipynb/html/Chapter%201%20-%20Mining%20Twitter.html

Neat Link - https://github.com/lennerd/TwitterGraph